# Load Libraries

In [16]:
import os
import json
import math
import random

import pandas as pd

from models import Model
from classes import  Satellite, Customer
from drawingmap import DrawingMap

# Functions

## Load Customers

In [2]:
def load_customers(DEBUG = False) -> list[Customer]:
    customers = []
    df = pd.read_csv('../others/Levantamiento de Información/customers_20210119.csv')
    # filtrado
    df = df[(df.category < 16000) & (df.cajas > 0)].reset_index().copy()
    df['isSmall'] = df.apply(lambda x: True if x['cajas']<=5 else False, axis=1)
    #
    for i in range(len(df)):
        new_customer = Customer(id=str(df.id_cliente[i]),
                                lon=df.loc[i, 'lon'],
                                lat=df.loc[i, 'lat'],
                                demand=df.loc[i, 'cajas'],
                                category=df.loc[i, 'category'],
                                isSmall=df.loc[i, 'isSmall']
                                )
        customers.append(new_customer)
    if DEBUG:
        print("-"*50)
        print("Count of customers: ", len(customers))
        print("First customer:")
        print(json.dumps(customers[0].__dict__,indent=2,default=str))
    return customers, df

## Load Satelites

In [3]:
def load_satellites(DEBUG = False) ->list[Satellite]:
    satellites = []
    df = pd.read_csv('../others/Levantamiento de Información/Informacion CD a Satelites.csv')
    df.fillna(0, inplace=True)
    #
    df['costFixed'] = [10000, 12000, 14000, 9000, 8000, 10000, 12000, 15000, 9000]
    df['costPerVehicle'] = [120, 140, 100, 90, 100, 120, 110, 100, 150]
    #
    for i in range(len(df)):
        newSatellite = Satellite(id=str(df.nombre[i]),
                               lon=df.longitud[i],
                               lat=df.latitud[i],
                               distanceFromDC=df.loc[i, 'distance.value']/1000,
                               durationFromDC=df.loc[i, 'duration.value']/3600,
                               durationInTrafficFromDC=df.loc[i, 'duration_in_traffic.value']/3600,
                               costFixed=df.loc[i, 'costFixed'],
                               costPerVehicle=df.loc[i, 'costPerVehicle'],
                               )
        satellites.append(newSatellite)
    if DEBUG:
        print("-"*50)
        print("Count of SATELLITES: ", len(satellites))
        print("First Satellite:")
        print(json.dumps(satellites[0].__dict__,indent=2,default=str))
    return satellites, df

## Load Matrixes: Distance and Durations with and without traffic

In [4]:
def distance(s: Satellite, c: Customer) -> float:
     return int(math.sqrt(math.pow(s.lat - c.lat, 2) + math.pow(s.lon - c.lon, 2))*1000)

In [5]:
def calculate_distance(satellites: list[Satellite], customers: list[Customer], DEBUG = False) -> dict[(str,str), float]:
    distance_matrix = dict(
        [((s.id, c.id), distance(s,c)) for s in satellites for c in customers]
    )
    if DEBUG:
        print("-"*50)
        print("Count - distance: ", len(distance_matrix))
    return distance_matrix

# Drawing Map

In [6]:
s, df_s = load_satellites(True)

--------------------------------------------------
Count of SATELLITES:  9
First Satellite:
{
  "id": "Abaroa",
  "lon": -68.128476,
  "lat": -16.512483,
  "geographyLocation": [
    -68.128476,
    -16.512483
  ],
  "distanceFromDC": 11.117,
  "durationFromDC": 0.5772222222222222,
  "durationInTrafficFromDC": 0.6044444444444445,
  "costFixed": "10000",
  "costPerVehicle": "120"
}


In [7]:
c, df_c = load_customers(True)

--------------------------------------------------
Count of customers:  1468
First customer:
{
  "id": "44821",
  "lon": -68.14302642,
  "lat": -16.50004048,
  "geographyLocation": [
    -68.14302642,
    -16.50004048
  ],
  "demand": 3.0,
  "category": "13024",
  "isSmall": "True"
}


In [8]:
distance_matrix = calculate_distance(s,c)

## La Paz with Customers

In [9]:
location_la_paz = [-16.501457, -68.149887]
drawer = DrawingMap(location_la_paz)
drawer.addCustomers(c)
map = drawer.viewMap()
map

In [10]:
# hue = drawer.setHue(13000,13100)
# drawer.addCustomers(c,label='category', radius=2, fill=False)
# map = drawer.viewMap()
# map

## Params for Model

In [11]:
capacity_small_vehicle = 30
capacity_large_vehicle = 120
cost_per_vehicle_dc = 200

In [20]:
# tarifa from dc
tarifa_from_dc = {}
for idx in df_c.id_cliente.unique():
    tarifa_from_dc[idx] = random.randint(1,10)

In [21]:
# tarifa from satellite
tarifa_from_satellite = distance_matrix

In [22]:
params = {
    'capacity_small_vehicle': capacity_small_vehicle,
    'capacity_large_vehicle': capacity_large_vehicle,
    'cost_per_vehicle': cost_per_vehicle_dc,
    'tariff_from_satellite': tarifa_from_satellite,
    'tariff_from_dc': tarifa_from_dc
}

# Model


In [ ]:
model_deterministic = Model()

In [ ]:
model_deterministic.build(c,
                          s,
                          capacity_small_vehicle=capacity_small_vehicle,
                          capacity_large_vehicle=capacity_large_vehicle,
                          cost_per_vehicle=cost_per_vehicle_dc,
                          tariff_from_satellite=tarifa_from_satellite,
                          tariff_from_dc=tarifa_from_dc)

In [ ]:
print(model_deterministic.optimizeModel())

In [ ]:
model_deterministic.model.getObjective().getValue()